## Embeddings

In the [Representing text as tensors](https://www.kaggle.com/code/aisuko/representing-text-as-tensors) notebook, we operated on high-dimensional bag-of-words vectors with the length of `vocab_size`, and we were explicity converting from low-dimentional positional representation vectors into sparse one-hot representations. Here is an example for embedding classifier:


The goal of using word embeddings and reducing the dimensionality are:
* Finding the meaning of words based on their word approximation to other words. This is done by taken two word vectors and analyzing how often the words in the vectors are used together. The higher the frequency, the more you can find a correlation and relationship between the words.
* This process of training the word embedding to find word approcimations between words in a given dimension is how we reduce the word representation of low-dimensions.
* Embedding vectors serve as numeric representations of words and are used as input to other machine learning network layers.
* The embedding vector becomes the stored lookup table for words in the vocabulary.

Here we are continue exploring the **News AG** dataset. To begin. let's load the data and get some definitions.

In [ ]:
import torch
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
from torchinfo import summary
import collections

In [ ]:
# check the paltform, Apple Silicon or Linux
import os, platform

torch_device="cpu"

if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    torch_device = 'cuda'
else:
    torch_device = 'mps' if platform.system() == 'Darwin' else 'cpu'

torch_device="cpu"

## Dealing with variable sequence size

When working with words, you are going to have text sequences or sentences that are of different lengths. This can be problematic in training the word embeddings neural network. For consistency in the word embedding and improve training performance, we would have to apply some padding. This can be done using the `torch.nn.functional.pad` on a tokenized dataset. It adds zero values to the empty indices at the end of the vector.

In [ ]:
# Loading dataset
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset, test_dataset = list(train_dataset), list(test_dataset)
classes = ['World','Sports','Business','Sci/Tech']

tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

## Building a vocabulary of all tokens

We first build the dictionary using the `Counter` object, and then create `Vocab` object that would help us deal with vectorization

In [ ]:
# Building vocab
def build_vocab(train_dataset,ngrams=1,min_freq=1):
    counter = collections.Counter()
    for (label, line) in train_dataset:
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line), ngrams=ngrams))
    vocab = torchtext.vocab.vocab(counter, min_freq=min_freq)
    return vocab

vocab = build_vocab(train_dataset,ngrams=1,min_freq=1)

In [ ]:
def encode(x, voc=None, unk=0,tokenizer=tokenizer):
    pass

def padify(b):
    # b is the list of tuples of length batch_size
    # - first element of a tuple = label
    # - second = feature (text, sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequnce in this minibatch
    l = max(map(len,v))
    return (
        # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)), mode='constant', value=0) for t in v])
    )

Let's use the first 2 sentences as example to view the text length difference and effects of padding.

In [ ]:
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = encode(first_sentence)
s_tokens = encode(second_sentence)

print(f'First sentence in dataset:\n{first_sentence}')
print('Length:', len(train_dataset[0][1]), '\n')
print(f'\nSecond sentence in dataset:\n{second_sentence}')
print('Length:', len(train_dataset[1][1]), '\n')

Let's use the text sequence from the news article headlines in our dataset to change into a tokenize vector. As you will see, the text sequence have different lengths. We will apply padding so all the text sequence wil have a fixed length. This approach is used when you have a large set of text sequences in your dataset.
* The length of the 1st and 2nd sentences displayed have difference lengths.
* The max length of the dataset tensors is the length of the longest sentence length in the entire dataset.
* The zeros are added to the empty indexes in the tensor.

In [ ]:
vocab_size =len(vocab)
labels, features = padify(train_dataset)
print(f'features:{features}')

print(f'\nlength of first sentence: {len(f_tokens)}')
print(f'length of second sentence: {len(s_tokens)}')
print(f'size of features: {features.size()}')


## What is embedding?

The idea of `embedding` is the process of mapping words into vectors, which reflects the `semantic meaning of a word`. The length of its vectors are the embedding dimensions size. We will later discuss how to build meningful word embeddings, but for now let's just think of embeddings as a way to lower dimensionality of a word vector.

So, embedding layer would take a word as an input, and produce an output vector of specified `embedding_size`. In a sense, it is very similar to `Linear` layer, but instead of taking one-hot encoded vector, it will be able to take a word number as an input.

By using embedding layer as a first layer in our network, we can switch from bag-or-words to `embedding bag` model, where we first convert each word in our text into corresponding embedding, and then compute some aggregate function over all those embeddings, such as `sum`, `avergae` or `max`.

<figure><img src="https://hostux.social/system/media_attachments/files/110/750/619/792/006/183/original/94496909c3a93b6e.png" alt="" width="1000"><figcaption><p>Source from Microsoft Learning </a> </p></figcaption></figure>


This time our classifier neural network will start with an embedding layer, then aggregation layer, and a linear classifier on top of it:
* `vocab_size` are the size of the total number of words we have in our vocabulary
* `embed_dim` are the length of the word dimensions that show relationship between words passed as in the network
* `num_class` are the number of new categories we are trying to classify (e.g. Word, Sports, Business, Sci/Tech)

In [ ]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = torch.nn.Linear(embed_dim, num_class)
    
    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1) # torch.mean() computes the mean of all elements in a tensor, it is a reduction operation
        return self.fc(x)

## Training embedding classifer

Now we'll define our training dataloader and use the `collate_fn` to apply the padify function to the datasets as they loaded in each batch. As a result, the training dataset will be padded.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=padify)

We can train the model using rh training function to run the embedding network. The training output serves as a vector lookup store based on the unique index tokens from the vocabulary.

In [ ]:
def train_eopch(net, dataloader, lr=0.01, optimizer=None, loss_fn=torch.nn.NLLLoss(), epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)
    net.train()
    total_loss, acc, count,i = 0,0,0,0
    for labels, features in dataloader:
        labels, features = labels.to(torch_device), features.to(torch_device)
        optimizer.zero_grad()
        output = net(features)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss
        _, predicted = torch.max(output, 1)
        acc += (predicted == labels).sum()
        count += len(labels)
        i+=1
        if i%report_freq == 0:
            print(f'iteration {count}, loss {total_loss.item()/count}, accuracy {acc.item()/count}') # item() is used to get the value of a tensor
        if epoch_size  and count >= epoch_size:
            break
    
    return total_loss.item()/count, acc.item()/count

In [ ]:
net = EmbedClassifier(vocab_size, embed_dim=32, num_class=len(classes)).to(torch_device)

train_eopch(net, train_loader,lr=1, epoch_size=1000)

>Note: We are only training for 25k records here (less than one full epoch) for the sake of time, but you can continue training, write a function to train for several epochs, and experiement with learning rate parameter to achieve higher accuracy. You should be able to go to the accuracy of about 90%.

## EmbeddingBag Layer and Variable_length Sequnce Representation

In the previous architecture, we need to pad all sequences to the same length in order to fit them into a minibatch. This is not the most efficient way to represent variable length sequynces - another approach would be to use `offset` vector, which would hold offsets to all sequences stored in one large vector.

<figure><img src="https://hostux.social/system/media_attachments/files/110/751/844/099/559/089/original/c67bdb8c6e8afa7f.png" alt="" width="1000"><figcaption><p>Source from Microsoft Learning </a> </p></figcaption></figure>

>Note: On the picture above, we show a sequence of characters, but in our example we are working with sequences of words. However, the general principle of representing sequences with offset vector remains the same.

To work with offset representation, we use PyTorch's `EmbeddingBag` layer. It is similar to `Embedding`, but it takes content vector and offset vector as input, and it also includes averaging layer, which can be `mean`, `sum` or `max`. Here is modified network that uses `EmbeddingBag` layer:


In [ ]:
class EmbedClassifier(torch,nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim, sparse=True) # sparse=True is used to save memory
        self.fc = torch.nn.Linear(embed_dim, num_class)
    
    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

To prepare the dataset for training, we need to provide a conversition funciton that will prepare the offset vector:

In [ ]:
def offsetify(b):
    # first, compute data tensot from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return (
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=offsetify)

The offset vector is calculated by first combining the sentences indices into one tensor sequence, then extracting the staring index location of each sentence in the sequence. For example:
* The length of the first sentence in our training dataset is 29. Meaning the first index of the offset will 0
* The length of the second sentence in the dataset is 42. Meaning the second index of the offset of will be 29, where the first sentence ended.
* The third index of the offset will be 29 +42=71, where the 2nd sentence ended.

In [ ]:
labels, features, offset = offsetify(train_dataset)
print(f'offset:{offset}')
print(f'\nlength of first sentence: {len(f_tokens)}')
print(f'length of second sentence: {len(s_tokens)}')
print(f'size of data vector: {features.size()}')
print(f'size of offset vector: {offset.size()}')